# GAN

## Libraries

In [ ]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from numpy.random import rand
from numpy.random import randn
import random
from sklearn.preprocessing import normalize
from tqdm import tqdm
from tensorflow.keras.layers import LeakyReLU

## Importing Data  

In [2]:
df = pd.read_excel(r'C:\Users\AMIRHOSEIN\Desktop\University\Data Mining\airline_train_BlanksRemoved.xlsx')
passengers_list = df.values.tolist()

satisfaction=[]
for i in range(len(passengers_list)):
    satisfaction.append(passengers_list[i][24])

data=np.array(passengers_list)[:,2:]
data,norms=normalize(data,norm='max',axis=0,return_norm=True)

In [5]:
norms

array([1.000e+00, 1.000e+00, 8.500e+01, 1.000e+00, 2.000e+00, 4.983e+03,
       5.000e+00, 5.000e+00, 5.000e+00, 5.000e+00, 5.000e+00, 5.000e+00,
       5.000e+00, 5.000e+00, 5.000e+00, 5.000e+00, 5.000e+00, 5.000e+00,
       5.000e+00, 5.000e+00, 1.592e+03, 1.584e+03, 1.000e+00])

## Functions

In [6]:
# generate n real samples with class labels
def values(array):
    s=[]
    for i in array:
        if i not in s:
            s.append(i)
#     s.sort()
    return np.array(s)

def generate_real_samples(data1):
    return data1,np.ones((data1.shape[0],1))


# define the standalone discriminator model
def define_discriminator(n_inputs=23):
    model = Sequential()
    model.add(Dense(50, activation='tanh', kernel_initializer='he_uniform', input_dim=n_inputs))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


# define the standalone generator model
def define_generator(latent_dim, n_outputs=23):
    model = Sequential()
    leaky_relu = LeakyReLU(alpha=1)
    model.add(Dense(32, activation=leaky_relu , input_dim=latent_dim))
    model.add(Dense(n_outputs, activation='sigmoid'))
    return model


# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n):
    # generate points in the latent space
    x_input = randn(latent_dim * n)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n, latent_dim)
    return x_input

# use the generator to generate n fake examples and plot the results
def generate_fake_samples1(generator, latent_dim, n):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n)
    # predict outputs
    X = generator.predict(x_input)
    # return the results
    return X,np.zeros((n,1))

# use the generator to generate n fake examples and plot the results
def generate_fake_samples2(data, n):
    X=np.zeros((n,data.shape[1]))
    for i in range(data.shape[1]):
        X[:,i] = random.choices(values(data[:,i]), k=n)
        
    return X.astype(int),np.zeros((n,1))

def generate_fake_samples3(data, n):
    X=rand(n,data.shape[1])
    return X,np.zeros((n,1))
    


# define the combined generator and discriminator model, for updating the generator
def define_gan(generator, discriminator):
    # make weights in the discriminator not trainable
    discriminator.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(generator)
    # add the discriminator
    model.add(discriminator)
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model


## Training & Producing fake data for discriminator

In [8]:
# create the discriminator
discriminator = define_discriminator()

In [9]:
# create the generator
generator = define_generator(latent_dim=50)

# generating real and fake data 
x_real, y_real = generate_real_samples(data)

# x_fake1, y_fake1 = generate_fake_samples1(generator, latent_dim=50 , n=int(data.shape[0]/2))
# x_fake2, y_fake2 = generate_fake_samples2(data , n=int(data.shape[0]/2))
x_fake, y_fake = generate_fake_samples3(data , n=int(data.shape[0]))
# x_fake=np.vstack((x_fake1,x_fake2))
# y_fake=np.vstack((y_fake1,y_fake2))


x=np.vstack((x_real,x_fake))
y=np.vstack((y_real,y_fake))

In [11]:
discriminator.fit(x,y,epochs=100)

Epoch 1/100
6475/6475 [==============================] - 6s 889us/step - loss: 0.0420 - accuracy: 0.9881
Epoch 2/100
6475/6475 [==============================] - 6s 877us/step - loss: 0.0154 - accuracy: 0.9952
Epoch 3/100
6475/6475 [==============================] - 6s 878us/step - loss: 0.0090 - accuracy: 0.9972
Epoch 4/100
6475/6475 [==============================] - 6s 876us/step - loss: 0.0053 - accuracy: 0.9985
Epoch 5/100
6475/6475 [==============================] - 6s 884us/step - loss: 0.0033 - accuracy: 0.9990
Epoch 6/100
6475/6475 [==============================] - 6s 891us/step - loss: 0.0021 - accuracy: 0.9994
Epoch 7/100
6475/6475 [==============================] - 6s 905us/step - loss: 0.0015 - accuracy: 0.9996
Epoch 8/100
6475/6475 [==============================] - 6s 875us/step - loss: 0.0011 - accuracy: 0.9997
Epoch 9/100
6475/6475 [==============================] - 6s 874us/step - loss: 8.9618e-04 - accuracy: 0.9998
Epoch 10/100
6475/6475 [===========================

In [12]:
_, acc_fake = discriminator.evaluate(x_fake, y_fake, verbose=0)
_, acc_real = discriminator.evaluate(x_real, y_real, verbose=0)
print(acc_fake,acc_real)

1.0 1.0


## Training GAN

In [13]:
gan=define_gan(generator,discriminator)
gan.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (None, 23)                2391      
                                                                 
 sequential (Sequential)     (None, 1)                 1251      
                                                                 
Total params: 3,642
Trainable params: 2,391
Non-trainable params: 1,251
_________________________________________________________________


In [14]:
x_gan = generate_latent_points(50,10001)
y_gan = np.ones((10001,1))
gan.fit(x_gan,y_gan,epochs=1000)


Epoch 1/1000
313/313 [==============================] - 1s 923us/step - loss: 19.8828
Epoch 2/1000
313/313 [==============================] - 0s 933us/step - loss: 2.9267
Epoch 3/1000
313/313 [==============================] - 0s 946us/step - loss: 0.4374
Epoch 4/1000
313/313 [==============================] - 0s 939us/step - loss: 0.1294
Epoch 5/1000
313/313 [==============================] - 0s 933us/step - loss: 0.0663
Epoch 6/1000
313/313 [==============================] - 0s 926us/step - loss: 0.0358
Epoch 7/1000
313/313 [==============================] - 0s 920us/step - loss: 0.0211
Epoch 8/1000
313/313 [==============================] - 0s 930us/step - loss: 0.0169
Epoch 9/1000
313/313 [==============================] - 0s 933us/step - loss: 0.0130
Epoch 10/1000
313/313 [==============================] - 0s 917us/step - loss: 0.0074
Epoch 11/1000
313/313 [==============================] - 0s 930us/step - loss: 0.0066
Epoch 12/1000
313/313 [==============================] - 0s 93

Epoch 94/1000
313/313 [==============================] - 0s 923us/step - loss: 8.3057e-04
Epoch 95/1000
313/313 [==============================] - 0s 943us/step - loss: 9.2093e-04
Epoch 96/1000
313/313 [==============================] - 0s 939us/step - loss: 3.1385e-04
Epoch 97/1000
313/313 [==============================] - 0s 917us/step - loss: 4.2539e-04
Epoch 98/1000
313/313 [==============================] - 0s 930us/step - loss: 5.0141e-04
Epoch 99/1000
313/313 [==============================] - 0s 930us/step - loss: 3.2990e-04
Epoch 100/1000
313/313 [==============================] - 0s 933us/step - loss: 9.7674e-04
Epoch 101/1000
313/313 [==============================] - 0s 926us/step - loss: 8.5815e-04
Epoch 102/1000
313/313 [==============================] - 0s 930us/step - loss: 3.1404e-04
Epoch 103/1000
313/313 [==============================] - 0s 923us/step - loss: 1.9219e-04
Epoch 104/1000
313/313 [==============================] - 0s 936us/step - loss: 6.0944e-06
Epoch

313/313 [==============================] - 0s 936us/step - loss: 9.7307e-08
Epoch 275/1000
313/313 [==============================] - 0s 926us/step - loss: 9.0072e-08
Epoch 276/1000
313/313 [==============================] - 0s 936us/step - loss: 8.2756e-08
Epoch 277/1000
313/313 [==============================] - 0s 917us/step - loss: 7.6359e-08
Epoch 278/1000
313/313 [==============================] - 0s 943us/step - loss: 7.0073e-08
Epoch 279/1000
313/313 [==============================] - 0s 914us/step - loss: 6.4663e-08
Epoch 280/1000
313/313 [==============================] - 0s 930us/step - loss: 5.9714e-08
Epoch 281/1000
313/313 [==============================] - 0s 930us/step - loss: 5.4880e-08
Epoch 282/1000
313/313 [==============================] - 0s 910us/step - loss: 5.0537e-08
Epoch 283/1000
313/313 [==============================] - 0s 923us/step - loss: 4.6548e-08
Epoch 284/1000
313/313 [==============================] - 0s 917us/step - loss: 4.3152e-08
Epoch 285/1000

313/313 [==============================] - 0s 936us/step - loss: 1.3479e-08
Epoch 455/1000
313/313 [==============================] - 0s 933us/step - loss: 1.2706e-08
Epoch 456/1000
313/313 [==============================] - 0s 930us/step - loss: 1.2257e-08
Epoch 457/1000
313/313 [==============================] - 0s 923us/step - loss: 1.1704e-08
Epoch 458/1000
313/313 [==============================] - 0s 927us/step - loss: 1.1054e-08
Epoch 459/1000
313/313 [==============================] - 0s 926us/step - loss: 1.0608e-08
Epoch 460/1000
313/313 [==============================] - 0s 926us/step - loss: 1.0237e-08
Epoch 461/1000
313/313 [==============================] - 0s 920us/step - loss: 9.9109e-09
Epoch 462/1000
313/313 [==============================] - 0s 939us/step - loss: 9.5437e-09
Epoch 463/1000
313/313 [==============================] - 0s 936us/step - loss: 9.2469e-09
Epoch 464/1000
313/313 [==============================] - 0s 920us/step - loss: 8.9283e-09
Epoch 465/1000

313/313 [==============================] - 0s 946us/step - loss: 2.7819e-07
Epoch 635/1000
313/313 [==============================] - 0s 933us/step - loss: 2.3202e-07
Epoch 636/1000
313/313 [==============================] - 0s 939us/step - loss: 1.9944e-07
Epoch 637/1000
313/313 [==============================] - 0s 946us/step - loss: 1.7428e-07
Epoch 638/1000
313/313 [==============================] - 0s 975us/step - loss: 1.5527e-07
Epoch 639/1000
313/313 [==============================] - 0s 939us/step - loss: 1.3991e-07
Epoch 640/1000
313/313 [==============================] - 0s 920us/step - loss: 1.2678e-07
Epoch 641/1000
313/313 [==============================] - 0s 936us/step - loss: 1.1601e-07
Epoch 642/1000
313/313 [==============================] - 0s 930us/step - loss: 1.0656e-07
Epoch 643/1000
313/313 [==============================] - 0s 946us/step - loss: 9.8234e-08
Epoch 644/1000
313/313 [==============================] - 0s 946us/step - loss: 9.0492e-08
Epoch 645/1000

313/313 [==============================] - 0s 956us/step - loss: 2.2093e-08
Epoch 815/1000
313/313 [==============================] - 0s 920us/step - loss: 2.0487e-08
Epoch 816/1000
313/313 [==============================] - 0s 920us/step - loss: 1.9003e-08
Epoch 817/1000
313/313 [==============================] - 0s 926us/step - loss: 1.7646e-08
Epoch 818/1000
313/313 [==============================] - 0s 917us/step - loss: 1.6362e-08
Epoch 819/1000
313/313 [==============================] - 0s 926us/step - loss: 1.5234e-08
Epoch 820/1000
313/313 [==============================] - 0s 923us/step - loss: 1.4157e-08
Epoch 821/1000
313/313 [==============================] - 0s 936us/step - loss: 1.3140e-08
Epoch 822/1000
313/313 [==============================] - 0s 943us/step - loss: 1.2205e-08
Epoch 823/1000
313/313 [==============================] - 0s 939us/step - loss: 1.1325e-08
Epoch 824/1000
313/313 [==============================] - 0s 933us/step - loss: 1.0618e-08
Epoch 825/1000

313/313 [==============================] - 0s 933us/step - loss: 1.5906e-07
Epoch 995/1000
313/313 [==============================] - 0s 939us/step - loss: 1.3685e-07
Epoch 996/1000
313/313 [==============================] - 0s 923us/step - loss: 1.1979e-07
Epoch 997/1000
313/313 [==============================] - 0s 930us/step - loss: 1.0544e-07
Epoch 998/1000
313/313 [==============================] - 0s 946us/step - loss: 9.3509e-08
Epoch 999/1000
313/313 [==============================] - 0s 930us/step - loss: 8.3707e-08
Epoch 1000/1000
313/313 [==============================] - 0s 949us/step - loss: 7.6115e-08


In [170]:
gan.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_21 (Sequential)  (None, 23)                2391      
                                                                 
 sequential_19 (Sequential)  (None, 1)                 1076      
                                                                 
Total params: 3,467
Trainable params: 2,391
Non-trainable params: 1,076
_________________________________________________________________


## Generating Data

In [16]:
Syn_data=np.zeros((10001,23))
for i in tqdm(range(10001)):
    Syn_data[i,:]=generator.predict(x_gan[i].reshape(1,50))

100%|████████████████████████████████████████████████████████████████████████████| 10001/10001 [06:43<00:00, 24.81it/s]


In [20]:
np.savetxt('Syn_data.csv', Syn_data[:-1,:], delimiter=',')